# <center> Lab 04: Multivariate Linear Regression 실습 <center/>

$$ H(x_1, x_2, x_3) = x_1w_1 + x_2w_2 + x_3w_3 + b $$

$$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $$

 - $H(x)$: 주어진 $x$ 값에 대해 예측을 어떻게 할 것인가
 - $cost(W, b)$: $H(x)$ 가 $y$ 를 얼마나 잘 예측했는가

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# For reproducibility
torch.manual_seed(1) # seed 고정

## Naive Data Representation

In [2]:
# 5명의 학생이 3번의 쪽지 시험 점수로 기말고사 점수를 예측

x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [3]:
# 모델 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w3.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.294 w2: 0.297 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.674 w2: 0.676 w3: 0.676 b: 0.008 Cost: 1.563634
Epoch  200/1000 w1: 0.679 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.497603
Epoch  300/1000 w1: 0.684 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.435026
Epoch  400/1000 w1: 0.689 w2: 0.678 w3: 0.678 b: 0.008 Cost: 1.375730
Epoch  500/1000 w1: 0.694 w2: 0.678 w3: 0.678 b: 0.009 Cost: 1.319503
Epoch  600/1000 w1: 0.699 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.266215
Epoch  700/1000 w1: 0.704 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.215693
Epoch  800/1000 w1: 0.709 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.167821
Epoch  900/1000 w1: 0.713 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.122419
Epoch 1000/1000 w1: 0.718 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.079375


cost가 점차 향상

## Matrix Data Representation

$$
\begin{pmatrix}
x_1 & x_2 & x_3
\end{pmatrix}
\cdot
\begin{pmatrix}
w_1 \\
w_2 \\
w_3 \\
\end{pmatrix}
=
\begin{pmatrix}
x_1w_1 + x_2w_2 + x_3w_3
\end{pmatrix}
$$

$$ H(X) = XW $$

In [4]:
# matrix로 변수를 정의

x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [6]:
# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)    # 기울기 W를 matrix로 정의하면, 코드 간결화
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b # or .mm or @

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712646
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371017
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost: 10.445305
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391228
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493135
Epoch   10/20 hypo

-------------------------

## `nn.Module`을 사용한 다중 선형 회귀 코드

In [7]:
# 다중 선형 회귀 클래스 정의

class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)   # 변수가 3개

    def forward(self, x):
        return self.linear(x)

In [8]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 31667.597656
Epoch    1/20 Cost: 9926.267578
Epoch    2/20 Cost: 3111.513672
Epoch    3/20 Cost: 975.451355
Epoch    4/20 Cost: 305.908539
Epoch    5/20 Cost: 96.042488
Epoch    6/20 Cost: 30.260748
Epoch    7/20 Cost: 9.641701
Epoch    8/20 Cost: 3.178671
Epoch    9/20 Cost: 1.152871
Epoch   10/20 Cost: 0.517863
Epoch   11/20 Cost: 0.318801
Epoch   12/20 Cost: 0.256388
Epoch   13/20 Cost: 0.236821
Epoch   14/20 Cost: 0.230660
Epoch   15/20 Cost: 0.228719
Epoch   16/20 Cost: 0.228095
Epoch   17/20 Cost: 0.227880
Epoch   18/20 Cost: 0.227799
Epoch   19/20 Cost: 0.227762
Epoch   20/20 Cost: 0.227732


-----------------------

## 데이터 로드하기

In [9]:
# 데이터 정의하기

from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

dataset = TensorDataset(x_train, y_train)

In [10]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 위 데이터 로더에서 dataset말고 인자를 2개 더 받았음
# batch_size 파이토치는 데이터를 정의할 때 batch size를 넣나봄 보통 2의 배수인데, 이는 하드웨워와 관련돼있다고 함
# shuffle은 순서를 섞어서 좀 더 로버스트한 결과를 기대할 수 있음

In [11]:
nb_epochs = 20

for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader): # suffle된 dataloader가 batch 단위로 반복됨
#     print(batch_idx)  
#     print(samples) # 어떤 데이터가 들어가는지
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

Epoch    0/20 Batch 1/3 Cost: 0.121737
Epoch    0/20 Batch 2/3 Cost: 0.020353
Epoch    0/20 Batch 3/3 Cost: 1.032858
Epoch    1/20 Batch 1/3 Cost: 0.057136
Epoch    1/20 Batch 2/3 Cost: 0.335384
Epoch    1/20 Batch 3/3 Cost: 0.976398
Epoch    2/20 Batch 1/3 Cost: 0.315299
Epoch    2/20 Batch 2/3 Cost: 0.362969
Epoch    2/20 Batch 3/3 Cost: 0.274226
Epoch    3/20 Batch 1/3 Cost: 0.042171
Epoch    3/20 Batch 2/3 Cost: 0.541584
Epoch    3/20 Batch 3/3 Cost: 0.010293
Epoch    4/20 Batch 1/3 Cost: 0.047489
Epoch    4/20 Batch 2/3 Cost: 0.089866
Epoch    4/20 Batch 3/3 Cost: 1.070885
Epoch    5/20 Batch 1/3 Cost: 0.393946
Epoch    5/20 Batch 2/3 Cost: 0.045611
Epoch    5/20 Batch 3/3 Cost: 0.409365
Epoch    6/20 Batch 1/3 Cost: 0.520396
Epoch    6/20 Batch 2/3 Cost: 0.149446
Epoch    6/20 Batch 3/3 Cost: 0.055988
Epoch    7/20 Batch 1/3 Cost: 0.590469
Epoch    7/20 Batch 2/3 Cost: 0.187555
Epoch    7/20 Batch 3/3 Cost: 0.050223
Epoch    8/20 Batch 1/3 Cost: 0.048731
Epoch    8/20 Batch 2/3 C

In [12]:
# 훈련된 모델

model

MultivariateLinearRegressionModel(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)

In [13]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 

# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 


훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[150.9652]], grad_fn=<AddmmBackward>)


### batch는 데이터를 전부 한바퀴 돌렸을 때가 1 epoch